In [1]:
from bokeh.plotting import figure, save, curdoc, output_file, show
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, CustomJS, HoverTool, LogColorMapper, GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper,callbacks
from bokeh.events import Tap 
from bokeh.layouts import widgetbox, column, row,gridplot
from bokeh.models.widgets import Button,CheckboxGroup
import geopandas as gpd
import pysal as ps
import pandas as pd
import numpy as np
import math
import coffeescript


In [31]:
import unittest



# Define a class in which the tests will run
class UnitTests(unittest.TestCase):

    # Each method in the class to execute a test
    def test_upper(self):
        self.assertEqual(1, 2)

    def test_isupper(self):
        self.assertEqual(1, 2)
        
def test(test_class=UnitTests):
    # Convenience function to run tests.
    # test_class is the class containing the tests.
    suite = unittest.TestLoader().loadTestsFromModule(test_class())
    unittest.TextTestRunner().run(suite)


In [61]:
# shapefile of Seattle zip codes
grid = r"../Data/zips_sea/shp.shp" 

#Shapefile of bus routes of Seattle
gridy = r"../Data/bus_seattle/network.shp"

#CSV file of zip codes and number of routes passing through them
zips_sea =pd.read_csv('../Data/zips_seattle.csv')

#CSV file of route numbers that passing through each zip code
zip_route =pd.read_csv('../Data/routes_zipcode.csv')

In [1]:
grid = gpd.read_file(grid)
network = gpd.read_file(network_fp)

NameError: name 'gpd' is not defined

In [65]:
# Extract x and y from geometry and have them as separate columns

def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]
    
def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon). 
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]
    
    # Check the geometry type
    gtype = geom.geom_type
    
    # "Normal" geometries
    # -------------------
    
    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )


grid['x'] = grid.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
grid['y'] = grid.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)


In [67]:
#Defining thresholds for income
breaks = [x for x in range(55000, 110000, 5000)]

#Initialize the classifier and apply it
classifier = ps.User_Defined.make(bins=breaks)
pt_classif = grid[['income']].apply(classifier)

# Rename the classified column
pt_classif.columns = ['incomeb']

# Join it back to the grid layer
grid = grid.join(pt_classif)

# Adding new column with bin names to be used in legend
grid['bin']= pd.np.where(grid.incomeb.astype(str) == '1', "[55000-60000]",pd.np.where(grid.incomeb.astype(str) == '2', "[60000-65000]",
                                                                                      pd.np.where(grid.incomeb.astype(str) == '3', "[65000-70000]",
                                                                                                 pd.np.where(grid.incomeb.astype(str) == '4', "[70000-75000]",
                                                                                                            pd.np.where(grid.incomeb.astype(str) == '5', "[75000-80000]",
                                                                                                                       pd.np.where(grid.incomeb.astype(str) == '6', "[80000-85000]",
                                                                                                                                  pd.np.where(grid.incomeb.astype(str) == '7', "[85000-90000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '8', "[90000-95000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '9', "[95000-100000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '10', "[100000-105000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '11', "[105000-110000]",'NA')
                        ))))))))))

C:\Users\ASUS\Anaconda3\lib\site-packages\pysal\esda\mapclassify.py:267: RuntimeWarning: invalid value encountered in greater
  binIds += (x > l) * (x <= r) * k
C:\Users\ASUS\Anaconda3\lib\site-packages\pysal\esda\mapclassify.py:267: RuntimeWarning: invalid value encountered in less_equal
  binIds += (x > l) * (x <= r) * k
C:\Users\ASUS\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4011: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [ ]:
#Sort shapefile based on income so have the legend in acsending order
grid = grid.sort_values(['income'])

In [68]:
#Drop the geometry from shapefile and create ColumnDataSource
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)

In [70]:
p = figure(title="Seattle Bus Routes")


In [72]:
type(gsource)

bokeh.models.sources.ColumnDataSource

In [71]:
class EntropyTest(unittest.TestCase):
    
    def test_plot(self):
        p = figure()
        self.assertEqual(str(type(p)),"<class 'bokeh.plotting.figure.Figure'>")

test(test_class=EntropyTest)

.
----------------------------------------------------------------------
Ran 1 test in 0.018s

OK


In [73]:
def js_code (N_plots):
    part1 = ''.join(['\n l'+str(i)+'.visible = '+'false;' for i in N_plots])
    part2 = ''.join(['\n if (cb_obj.active[i] == '+str(i)+')'+'{l'+str(i)+'.visible = '+'true;' +  '} else ' for i in N_plots])
    if part2.endswith('else '):
        part2 = part2[:-5]
    checkbox_code = '//console.log(cb_obj.active); '+ part1 + """for (i in cb_obj.active) {//console.log(cb_obj.active[i]);""" + part2 + """}""" 
    return (checkbox_code)

code = js_code (range(0,3))

def extract_geom (x):
    L=x.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
    return(L)
extract_geom (zips_sea)  
    

KeyError: ('geometry', 'occurred at index 0')

In [74]:
type(code)

str

In [60]:
class EntropyTest(unittest.TestCase):
    def test_getXY (self):
        self.assertRaises(AttributeError,extract_geom,zips_sea)
        self.assertRaises(KeyError,extract_geom,zips_sea)
    def test_plot(self):
        p = figure()
        self.assertEqual(str(type(p)),"<class 'bokeh.plotting.figure.Figure'>")

test(test_class=EntropyTest)

E
ERROR: test_getXY (__main__.EntropyTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2483, in get_value
    return libts.get_value_box(s, key)
  File "pandas/_libs/tslib.pyx", line 923, in pandas._libs.tslib.get_value_box (pandas\_libs\tslib.c:18843)
  File "pandas/_libs/tslib.pyx", line 932, in pandas._libs.tslib.get_value_box (pandas\_libs\tslib.c:18477)
TypeError: 'str' object cannot be interpreted as an integer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-60-dc42c40197ea>", line 3, in test_getXY
    self.assertRaises(AttributeError,extract_geom,zips_sea)
  File "C:\Users\ASUS\Anaconda3\lib\unittest\case.py", line 733, in assertRaises
    return context.handle('assertRaises', args, kwargs)
  File "C:\Users\ASUS\Anaconda3\lib\unittest\case.py", line 178, i

In [20]:
test()

0     POLYGON ((-122.340035 47.61306099999999, -122....
1     POLYGON ((-122.327661 47.649237, -122.32697 47...
2     POLYGON ((-122.35772 47.671873, -122.357721 47...
3     POLYGON ((-122.332911 47.606086, -122.333564 4...
4     POLYGON ((-122.328271 47.668646, -122.328259 4...
5     POLYGON ((-122.36573 47.53987799999999, -122.3...
6     POLYGON ((-122.405716 47.66858, -122.406027 47...
7     POLYGON ((-122.3353 47.554184, -122.335292 47....
8     POLYGON ((-122.356779 47.632315, -122.357017 4...
9     POLYGON ((-122.315539 47.631864, -122.314758 4...
10    POLYGON ((-122.326291 47.686799, -122.326649 4...
11    POLYGON ((-122.42139 47.580768, -122.413278 47...
12    POLYGON ((-122.393248 47.701602, -122.391995 4...
13    POLYGON ((-122.29156 47.553456, -122.291557 47...
14    POLYGON ((-122.380217 47.648463, -122.380422 4...
15    POLYGON ((-122.352841 47.61859, -122.351362 47...
16    POLYGON ((-122.32076 47.61040999999999, -122.3...
17    POLYGON ((-122.328803 47.715892, -122.3288